### モデルの作成
---
参考：
- [all](http://ni4muraano.hatenablog.com/entry/2017/08/10/101053)
- [batch normalization](https://www.kaggle.com/dingdiego/u-net-batchnorm-augmentation-stratification)

In [1]:
from keras.models import Model
from keras.layers.convolutional import Conv2D, ZeroPadding2D, Conv2DTranspose
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, ZeroPadding2D, ZeroPadding3D, AveragePooling3D
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout
from keras.layers.merge import concatenate


class DenceUNet(object):
    def __init__(self, input_channel_count, output_channel_count, first_layer_filter_count):
        self.INPUT_IMAGE_SIZE = 512
        self.CONCATENATE_AXIS = -1
        self.CONV_FILTER_SIZE = 4
        self.CONV_STRIDE = 2
        self.CONV_PADDING = (1, 1)
        self.DECONV_FILTER_SIZE = 2
        self.DECONV_STRIDE = 2

        # (512 x 512 x input_channel_count)
        inputs = Input((self.INPUT_IMAGE_SIZE, self.INPUT_IMAGE_SIZE, input_channel_count))
        conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
        conv11 =  BatchNormalization()(conv11)
        conc11 = concatenate([inputs, conv11], axis=3)
        conv12 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc11)
        conv12 =  BatchNormalization()(conv12)
        conc12 = concatenate([inputs, conv12], axis=3)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conc12)
        
        conv21 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
        conv21 =  BatchNormalization()(conv21)
        conc21 = concatenate([pool1, conv21], axis=3)
        conv22 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc21)
        conv22 =  BatchNormalization()(conv22)
        conc22 = concatenate([pool1, conv22], axis=3)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conc22)
        
        conv31 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
        conv31 =  BatchNormalization()(conv31)
        conc31 = concatenate([pool2, conv31], axis=3)
        conv32 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc31)
        conv32 =  BatchNormalization()(conv32)
        conc32 = concatenate([pool2, conv32], axis=3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conc32)
        
        conv41 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
        conv41 =  BatchNormalization()(conv41)
        conc41 = concatenate([pool3, conv41], axis=3)
        conv42 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc41)
        conv42 =  BatchNormalization()(conv42)
        conc42 = concatenate([pool3, conv42], axis=3)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conc42)
        
        conv51 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
        conv51 =  BatchNormalization()(conv51)
        conc51 = concatenate([pool4, conv51], axis=3)
        conv52 = Conv2D(512, (3, 3), activation='relu', padding='same')(conc51)
        conv52 =  BatchNormalization()(conv52)
        conc52 = concatenate([pool4, conv52], axis=3)

        
        up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conc52), conc42], axis=3)
        conv61 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
        conv61 =  BatchNormalization()(conv61)
        conc61 = concatenate([up6, conv61], axis=3)
        conv62 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc61)
        conv62 =  BatchNormalization()(conv62)
        conc62 = concatenate([up6, conv62], axis=3)

        up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conc62), conv32], axis=3)
        conv71 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
        conv71 =  BatchNormalization()(conv71)
        conc71 = concatenate([up7, conv71], axis=3)
        conv72 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc71)
        conv72 =  BatchNormalization()(conv72)
        conc72 = concatenate([up7, conv72], axis=3)

        up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conc72), conv22], axis=3)
        conv81 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
        conv81 =  BatchNormalization()(conv81)
        conc81 = concatenate([up8, conv81], axis=3)
        conv82 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc81)
        conv82 =  BatchNormalization()(conv82)
        conc82 = concatenate([up8, conv82], axis=3)

        up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conc82), conv12], axis=3)
        conv91 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
        conv91 =  BatchNormalization()(conv91)
        conc91 = concatenate([up9, conv91], axis=3)
        conv92 = Conv2D(32, (3, 3), activation='relu', padding='same')(conc91)
        conv92 =  BatchNormalization()(conv92)
        conc92 = concatenate([up9, conv92], axis=3)

        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conc92)
        
        self.DenceUNET = Model(inputs=[inputs], outputs=[conv10])
        

    def _add_encoding_layer(self, filter_count, sequence):
        new_sequence = LeakyReLU(0.2)(sequence)
        new_sequence = ZeroPadding2D(self.CONV_PADDING)(new_sequence)
        new_sequence = Conv2D(filter_count, self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        return new_sequence

    def _add_decoding_layer(self, filter_count, add_drop_layer, sequence):
        new_sequence = Activation(activation='relu')(sequence)
        new_sequence = Conv2DTranspose(filter_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE,
                                       kernel_initializer='he_uniform')(new_sequence)
        new_sequence = BatchNormalization()(new_sequence)
        if add_drop_layer:
            new_sequence = Dropout(0.5)(new_sequence)
        return new_sequence

    def get_model(self):
        return self.DenceUNET

### その他関数定義
---

In [2]:
import cv2
IMAGE_SIZE = 512
TRAIN_PERCENTAGE = 0.8

# 値を-1から1に正規化する関数
def normalize_x(image):
    # image = image/127.5 - 1
    image = (image-np.min(image)) / (np.max(image)/2) - 1
    return image


# 値を0から1に正規化する関数
def normalize_y(image):
    image = image/255
    return image


# 値を0から255に戻す関数
def denormalize_y(image):
    image = image*255
    return image


# インプット画像を読み込む関数
def load_X(folder_path, mode='train'):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*.mhd'))
    limits = int(len(mhd_files)*TRAIN_PERCENTAGE)

    if mode=='test':
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[limits].split('/')[-1].split('.')[0] +'-'+str(i))    

        for mhd_name in mhd_files[(int(limits)+1):]:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))  
                
    else:
        images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
        
        image_files = []
        for i in range(images.shape[0]):
            image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    

        for mhd_name in mhd_files[1:int(limits)]:
            mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
            images = np.concatenate([images,mhd_array])
            for i in range(mhd_array.shape[0]):
                image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     
    
    images = normalize_x(images[:, :, :, np.newaxis])
    return images, image_files


# ラベル画像を読み込む関数
def load_Y(folder_path):
    import glob, SimpleITK
    from natsort import natsorted
    
    mhd_files = natsorted(glob.glob(folder_path+'*/*[!2].mhd'))
    limits = int(len(mhd_files)*TRAIN_PERCENTAGE)
    
    images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
    for mhd_name in mhd_files[1:int(limits)]:
        mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
        images = np.concatenate([images,mhd_array])
    
    # GTの値の範囲が0-6だったので、1以上は1にしている
    images = np.where(images >= 1 , 1 ,images)
    return images[:, :, :, np.newaxis]


### train/test部分
---

参考：
- [クロスバリデーション](https://lp-tech.net/articles/Y56uo)
- [モデルチェックポイント](https://blog.shikoan.com/keras-model-checkpoint-save-best-only/)
- [その全て](https://keras.io/ja/callbacks/)

- [multi GPU](http://tech.wonderpla.net/entry/2018/01/09/110000)

In [3]:
from keras import backend as K
from keras.utils import multi_gpu_model #GPU 2枚用

# GPU1つのみの設定
if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "1"
set_session(tf.Session(config=config))

In [9]:
import os
import glob
from natsort import natsorted
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from sklearn.model_selection import KFold, train_test_split


# ダイス係数を計算
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return 2.0 * intersection / (K.sum(y_true) + K.sum(y_pred) + 1)

# ロス関数
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# denceU-Netのトレーニングを実行する関数
def train_denceunet():
    
    datasets = natsorted(glob.glob('../Preprocess/mhd/*'))
    X_train, file_names = load_X('../Preprocess/mhd/', 'train')
    Y_train = load_Y('../Preprocess/GroundTruth/')
    
    # in:1チャンネル out:1チャンネル
    input_channel_count = 1
    output_channel_count = 1

    # 一番初めのConvolutionフィルタ枚数は64
    first_layer_filter_count = 64
    BATCH_SIZE = 2
    NUM_EPOCH = 30

    # denceU-Netの生成
    network = DenceUNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), metrics=['accuracy', dice_coef], loss=dice_coef_loss )
    #model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), loss='binary_crossentropy', metrics=['accuracy'] )
    #model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=['accuracy', dice_coef] )
    
    plot_model(model, to_file='model.png')   
    model.summary()
    
    # GPU 2枚用
    # parallel_model = multi_gpu_model(model, gpus=2)
    # parallel_model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), metrics=[dice_coef], loss=dice_coef_loss )
    # parallel_model.summary()
    
       
    
    # datasetsの数だけ回して、train/testの組み合わせを検証
    # for i in range(len(datasets)):
        # X_train, file_names = load_X('../Preprocess/mhd/', 'train')   
    
    # K-fold split validationデータの組み合わせ検証
    kf = KFold(n_splits=12, shuffle=False)
    for i, (train_index, eval_index) in enumerate(kf.split(X_train)):
        X_tra, X_eval = X_train[train_index], X_train[eval_index]
        Y_tra, Y_eval = Y_train[train_index], Y_train[eval_index]         

        cp = ModelCheckpoint("weights/weights-"+str(i)+"crs-{epoch:02d}ep-{val_loss:.2f}loss.hdf5",
                             monitor="val_loss", verbose=1, save_best_only=True, save_weights_only=False)
        learning_history = model.fit(X_tra, Y_tra, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, callbacks=[cp], validation_data=(X_eval, Y_eval))
        model.save_weights('weights/weights-ver'+ str(i) +'.hdf5')
        
        # GPU 2枚用
        # learning_history = parallel_model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, callbacks=[cp], validation_data=(X_eval, y_eval))
        # parallel_model.save_weights('weights/weights-ver'+ str(i) +'.hdf5')  # <- ここだけ未確認(model.saveでok?)

        
        plt.plot(range(1, NUM_EPOCH+1), learning_history.history['dice_coef'], label="training")
        plt.plot(range(1, NUM_EPOCH+1), learning_history.history['val_dice_coef'], label="validation")
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.ylim(0,1)
        plt.grid(axis='y', color='lightgray')
        plt.savefig('progress-crs#'+str(i)+'.png')
        plt.clf()
    

# 学習後のdenceU-Netによる予測を行う関数
def predict():
    import cv2

    # testDataフォルダ配下にleft_imagesフォルダを置いている
    X_test, file_names = load_X('../Preprocess/mhd/', 'test')

    input_channel_count = 1
    output_channel_count = 1
    first_layer_filter_count = 64
    network = DenceUNet(input_channel_count, output_channel_count, first_layer_filter_count)
    model = network.get_model()
    model.load_weights('weights/weights-0crs-11ep-0.02loss.hdf5')
    BATCH_SIZE = 8
    Y_pred = model.predict(X_test, BATCH_SIZE)
    
    # 変な方向の断面図でないか、気になる！
    print(Y_pred.shape)
    
    # testDataフォルダ配下にleft_imagesフォルダを置いている
    save_path = 'DenceU-Net_DataSet/pred/'
    if not os.path.exists(save_path): os.makedirs(save_path)

    for i, y in enumerate(Y_pred):
        cv2.imwrite(save_path+'prediction' + file_names[i] + '.png', denormalize_y(y))


#if __name__ == '__main__':
#     train_denceunet()
#     predict()

In [10]:
train_denceunet()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 512, 512, 32) 320         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_37 (BatchNo (None, 512, 512, 32) 128         conv2d_39[0][0]                  
__________________________________________________________________________________________________
concatenate_45 (Concatenate)    (None, 512, 512, 33) 0           input_3[0][0]                    
                                                                 batch_normalization_37[0][0]     
__________

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dlbox/anaconda3/envs/TF2-NT/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-7306309ceda8>", line 1, in <module>
    train_denceunet()
  File "<ipython-input-9-b8ae64296446>", line 71, in train_denceunet
    learning_history = model.fit(X_tra, Y_tra, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, callbacks=[cp], validation_data=(X_eval, Y_eval))
  File "/home/dlbox/anaconda3/envs/TF2-NT/lib/python3.6/site-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/home/dlbox/anaconda3/envs/TF2-NT/lib/python3.6/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/home/dlbox/anaconda3/envs/TF2-NT/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/home/dlbox/anacond

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
predict()

In [11]:
import glob
from natsort import natsorted
natsorted(glob.glob('../Preprocess/mhd/*'))

['../Preprocess/mhd/L0.mhd',
 '../Preprocess/mhd/L0.raw',
 '../Preprocess/mhd/L1.mhd',
 '../Preprocess/mhd/L1.raw',
 '../Preprocess/mhd/L3.mhd',
 '../Preprocess/mhd/L3.raw',
 '../Preprocess/mhd/L4.mhd',
 '../Preprocess/mhd/L4.raw',
 '../Preprocess/mhd/M0.mhd',
 '../Preprocess/mhd/M0.raw',
 '../Preprocess/mhd/M1.mhd',
 '../Preprocess/mhd/M1.raw',
 '../Preprocess/mhd/M2.mhd',
 '../Preprocess/mhd/M2.raw',
 '../Preprocess/mhd/M3.mhd',
 '../Preprocess/mhd/M3.raw',
 '../Preprocess/mhd/M4.mhd',
 '../Preprocess/mhd/M4.raw',
 '../Preprocess/mhd/M5.mhd',
 '../Preprocess/mhd/M5.raw',
 '../Preprocess/mhd/S0.mhd',
 '../Preprocess/mhd/S0.raw',
 '../Preprocess/mhd/S1.mhd',
 '../Preprocess/mhd/S1.raw',
 '../Preprocess/mhd/S2.mhd',
 '../Preprocess/mhd/S2.raw',
 '../Preprocess/mhd/S3.mhd',
 '../Preprocess/mhd/S3.raw',
 '../Preprocess/mhd/S4.mhd',
 '../Preprocess/mhd/S4.raw']